In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
s3 = boto3.resource('s3')
buckets = s3.Bucket('deutsche-boerse-xetra-pds')

In [3]:
arg_date = '2021-05-07'
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)
arg_date_dt

datetime.date(2021, 5, 6)

In [4]:
objects = [obj for obj in buckets.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt]

In [5]:
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [6]:
csv_obj = buckets.Object(key='2021-03-15/2021-03-15_BINS_XETR15.csv').get().get('Body').read().decode('utf-8')

In [7]:
data= StringIO(csv_obj)
df_init= pd.read_csv(data, delimiter=',')
columns = df_init.columns

In [8]:
df_all = pd.DataFrame(columns=columns)

In [9]:
for obj in objects:
    csv_obj = buckets.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data= StringIO(csv_obj)
    df= pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
df_all

In [ ]:
columns = ['ISIN', 'Mnemonic',  'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
df_all = df_all[columns]
df_all

In [ ]:
df_all.dropna(inplace=True)
df_all.shape

In [ ]:
df_all['opening_price'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform('first')
df_all

In [ ]:
df_all[df_all.Mnemonic == 'S92']

In [ ]:
df_all['closing_price'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform('last')
df_all

In [ ]:
df_test = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price=('opening_price','min'),
                                                             closing_price_eur=('closing_price','min'),
                                                             minimum_price_eur=('MinPrice','min'),
                                                             maximum_price_eur=('MaxPrice','max'),
                                                             daily_traded_volumn=('TradedVolume','sum'))
df_test

In [ ]:
df_test['prev_closing_price'] = df_test.sort_values('Date').groupby('ISIN')['closing_price_eur'].shift(1)
df_test

In [ ]:
df_test['change_prev_closing_%'] = (df_test.closing_price_eur - df_test.prev_closing_price)/df_test.closing_price_eur*100
df_test.drop(columns='prev_closing_price', inplace=True)
df_test = df_test.round(decimals = 2)
df_test = df_test[df_test.Date >= arg_date]
df_test.head(10)

In [ ]:
key = 'xetra_daily_report' + datetime.today().strftime('%Y%m%d-%H%M%S') + '.parquet'

In [ ]:
out_buffer = BytesIO()
df_test.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('s3-udemy-ichih')
bucket_target.put_object(Body=out_buffer.getvalue(),Key=key)

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_target.Object(key='??').get('Body').read()
data= BytesIO(prq_obj)
df_report = pd.read_parquet(data)